<a href="https://colab.research.google.com/github/SkyRanger2010/DE2024_PY/blob/main/HomeWork_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kafka

In [ ]:
import random
import time
from kafka import KafkaProducer
import json
from datetime import datetime

actions = ["login", "logout", "purchase", "click"]
data = []

for user_id in range(1, 101):  # 100 пользователей
    for _ in range(random.randint(1, 10)):  # Случайное количество действий от 1 до 10
        action = random.choice(actions)
        timestamp = datetime.now().isoformat()
        data.append({"user_id": user_id, "action": action, "timestamp": timestamp})

# Запись данных в файл
with open('client_actions.json', 'w') as f:
    json.dump(data, f)

producer = KafkaProducer(
    bootstrap_servers='localhost:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8'),
    acks='all',
    retries=5,
    linger_ms=10
)

with open('client_actions.json') as f:
    data = json.load(f)
    for record in data:
        future = producer.send('example_topic', record)

producer.flush()
producer.close()

In [ ]:
from kafka import KafkaConsumer
from collections import Counter
import json

consumer = KafkaConsumer('example_topic',
                         bootstrap_servers='localhost:9094',
                         auto_offset_reset='earliest',
                         enable_auto_commit=True,
                         group_id='my-group',
                         value_deserializer=lambda x: json.loads(x.decode('utf-8')))

action_counter = Counter()

for message in consumer:
    action = message.value['action']
    action_counter[(message.value['user_id'], action)] += 1

# Получаем пользователей с наибольшим количеством действий
most_active_users = action_counter.most_common()

# Выводим результаты
for (user_id, action), count in most_active_users:
    print(f'User ID: {user_id}, Action: {action}, Count: {count}')

consumer.close()